In [ ]:
!pip install langsmith

In [ ]:
!pip freeze

In [ ]:
!pip install chromadb.

In [ ]:
!pip install langchain-community 

In [ ]:
!pip install sentence-transformers

In [ ]:
import json
from uuid import uuid4

def make_chunks(chat_data):
    chunks = []
    i = 0
    while i < len(chat_data):
        if chat_data[i]["speaker"] == "Human":
            human_msg = chat_data[i]["message"]
            ai_msg = chat_data[i + 1]["message"] if (i + 1 < len(chat_data) and chat_data[i + 1]["speaker"] == "AI") else ""
            
            chunk = {
                "id": f"turn_{i}_{uuid4().hex[:6]}",
                "text": human_msg.strip(),
                "metadata": {
                    "speaker": "human",
                    "ai_reply": ai_msg.strip(),
                    "turn_number": i,
                    "emotion": None,   # placeholder for future tagging
                    "topic": None
                }
            }
            chunks.append(chunk)
        i += 1
    return chunks


# Example
with open("chat.json", "r") as f:
    chat_data = json.load(f)

chunks = make_chunks(chat_data)

with open("chunks_ready.json", "w") as f:
    json.dump(chunks, f, indent=2)

print(f"Created {len(chunks)} chunks ✅")


In [9]:
import json
with open("chunks_ready.json", "r") as f:
    chunks = json.load(f)

print(f"Loaded {len(chunks)} chunks ✅")
print(chunks[0])

Loaded 25 chunks ✅
{'id': 'turn_0_430ba2', 'text': 'I’ve been feeling off lately, like I’m just going through the motions every day.', 'metadata': {'speaker': 'human', 'ai_reply': 'That sounds really hard. When life starts feeling repetitive or dull, it’s often a sign your mind needs a break or some new stimulus. Can you tell me what’s been draining you lately?', 'turn_number': 0, 'emotion': 'test', 'topic': 'test'}}


In [11]:
import os
import json
from uuid import uuid4
import chromadb
from sentence_transformers import SentenceTransformer

# ---------- STEP 3: Initialize persistent Chroma ----------
# Create persistent directory
persist_dir = "./chroma_store"
os.makedirs(persist_dir, exist_ok=True)

# Persistent client
client = chromadb.PersistentClient(path=persist_dir)

# Create or get collection
collection = client.get_or_create_collection("mental_health_chat")
print("✅ Connected to persistent Chroma collection.")


# ---------- STEP 4: Generate embeddings ----------
model = SentenceTransformer("all-MiniLM-L6-v2")

ids = [c["id"] for c in chunks]
texts = [c["text"] for c in chunks]
metadatas = [c["metadata"] for c in chunks]

embeddings = model.encode(texts).tolist()

# ---------- STEP 5: Add to Chroma (if not already present) ----------
existing_ids = set(collection.get(ids=ids).get("ids", []))
new_chunks = [(i, t, m, e) for i, t, m, e in zip(ids, texts, metadatas, embeddings) if i not in existing_ids]

if new_chunks:
    collection.add(
        ids=[i for i, _, _, _ in new_chunks],
        documents=[t for _, t, _, _ in new_chunks],
        metadatas=[m for _, _, m, _ in new_chunks],
        embeddings=[e for _, _, _, e in new_chunks],
    )
    print(f"✅ Added {len(new_chunks)} new chunks to persistent Chroma.")
else:
    print("ℹ️ No new chunks to add. Database already up-to-date.")


# ---------- STEP 6: Example Query ----------
query = "I keep feeling tired and unmotivated lately."
query_emb = model.encode([query]).tolist()

results = collection.query(query_embeddings=query_emb, n_results=1, include=["documents", "metadatas", "distances"])

print("\n🔍 Top Matches:")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"\n🗣️ Human said: {doc}")
    print(f"🤖 AI replied: {meta['ai_reply']}")
    print("-" * 60)

✅ Connected to persistent Chroma collection.
ℹ️ No new chunks to add. Database already up-to-date.

🔍 Top Matches:

🗣️ Human said: I’ve been feeling off lately, like I’m just going through the motions every day.
🤖 AI replied: That sounds really hard. When life starts feeling repetitive or dull, it’s often a sign your mind needs a break or some new stimulus. Can you tell me what’s been draining you lately?
------------------------------------------------------------


In [ ]:
vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embeddings)
retriever = vectorstore.as_retriever()


In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key='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')
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)
print(prompt)